# Pinhole Camera Model Practice Questions (Solutions)

This notebook provides the solutions to the practice questions. The solutions use the `numpy` library for cleaner mathematical operations.

In [1]:
import numpy as np

---

## Question 1: Perspective Projection

A 3D point in the camera coordinate system is given by `p_c = [10, 5, 20]`. The camera has a focal length `f` of 50mm. Calculate the position of the projection of this point on the image plane in meters.

### Solution
We use the perspective projection equations:
$$ u_m^c = f \frac{p_x^c}{p_z^c} $$ 
$$ v_m^c = f \frac{p_y^c}{p_z^c} $$ 
First, we convert the focal length from millimeters to meters: `f = 50mm = 0.05m`.

In [2]:
# Define the variables
p_c = np.array([10, 5, 20])
f = 0.05 # in meters

# Apply the perspective projection formulas
u_m_c = f * (p_c[0] / p_c[2])
v_m_c = f * (p_c[1] / p_c[2])

p_m_c = np.array([u_m_c, v_m_c])
print(f"Projected point (in meters): {p_m_c}")

Projected point (in meters): [0.025  0.0125]


---

## Question 2: Conversion to Pixels

Given the projected point from Question 1, convert its coordinates from meters to pixels. Assume the following intrinsic parameters: `s_x = 800` pixels/meter, `s_y = 800` pixels/meter, `o_x = 320` pixels, `o_y = 240` pixels. What are the pixel coordinates `(u^I, v^I)`?

### Solution
We use the conversion formulas:
$$ u^I = s_x u_m^c + o_x $$
$$ v^I = s_y v_m^c + o_y $$

In [3]:
# Define intrinsic parameters
s_x = 800
s_y = 800
o_x = 320
o_y = 240

# Use the projected point from Q1
# p_m_c = np.array([0.025, 0.0125])

# Apply the conversion formulas
u_I = s_x * p_m_c[0] + o_x
v_I = s_y * p_m_c[1] + o_y

p_I = np.array([u_I, v_I])
print(f"Pixel coordinates: {p_I}")

Pixel coordinates: [340. 250.]


---

## Question 3: Intrinsic Matrix

Construct the intrinsic calibration matrix `K` using the parameters from Question 2, assuming no skew (`s_θ = 0`).

### Solution
The intrinsic matrix `K` is defined as:
$$ K = \begin{bmatrix} f s_x & s_\theta & o_x \\ 0 & f s_y & o_y \\ 0 & 0 & 1 \end{bmatrix} = \begin{bmatrix} \alpha_x & s_\theta & o_x \\ 0 & \alpha_y & o_y \\ 0 & 0 & 1 \end{bmatrix} $$

In [4]:
# Parameters from previous questions
# f = 0.05, s_x = 800, s_y = 800, o_x = 320, o_y = 240
s_theta = 0

# Calculate alpha_x and alpha_y
alpha_x = f * s_x
alpha_y = f * s_y

# Construct the matrix K
K = np.array([
    [alpha_x, s_theta, o_x],
    [0,       alpha_y, o_y],
    [0,       0,       1]
])

print("Intrinsic Matrix K:")
print(K)

Intrinsic Matrix K:
[[ 40.   0. 320.]
 [  0.  40. 240.]
 [  0.   0.   1.]]


---

## Question 4: Point Projection with Known Pose

A point in the world frame is given by `p_w = [2, 3, 50]`. The camera's pose `T_w^c` is given by an identity rotation matrix and a translation vector `t = [0, 0, 10]`. Using the intrinsic matrix `K` from Question 3, calculate the pixel coordinates of the projected point.

### Solution
The full projection equation is:
$$ \lambda \tilde{p}^I = K \, p^c = K (R \, p^w + t) $$
Where `λ` is the depth. To get the final pixel coordinates, we perform the matrix-vector multiplication and then divide the first two components of the result by the third.

In [5]:
# Define world point and pose
p_w = np.array([2, 3, 50])
R = np.identity(3)
t = np.array([0, 0, 10])

# 1. Transform world point to camera coordinates
p_c_q4 = R @ p_w + t
print(f"Point in camera coordinates: {p_c_q4}")

# 2. Project using intrinsic matrix K
p_I_homogeneous = K @ p_c_q4
print(f"Projected point (homogeneous): {p_I_homogeneous}")

# 3. Convert from homogeneous to 2D coordinates by dividing by the last element
u_I_final = p_I_homogeneous[0] / p_I_homogeneous[2]
v_I_final = p_I_homogeneous[1] / p_I_homogeneous[2]

print(f"Final Pixel Coordinates (u, v): [{u_I_final:.3f} {v_I_final:.3f}]")

Point in camera coordinates: [ 2.  3. 60.]
Projected point (homogeneous): [19280. 14520.    60.]
Final Pixel Coordinates (u, v): [321.333 242.000]


---

## Question 5: Radial Distortion

A point is projected on the image plane at `(u_distort^I, v_distort^I) = (400, 350)` including radial distortion. Given distortion coefficients `a1 = 0.0001` and `a2 = 0.0000002` and the principal point `(o_x, o_y) = (320, 240)`, calculate the corrected pixel coordinates `(u^I, v^I)`.

### Solution
The formulas to find the corrected coordinates `(u^c, v^c)` from the distorted ones `(u_distort^c, v_distort^c)` are:
$$ u^{c}=(1+a_{1}r^{2}+a_{2}r^{4})u_{distort}^{c} $$
$$ v^{c}=(1+a_{1}r^{2}+a_{2}r^{4})v_{distort}^{c} $$ 
where $r^{2}=(u_{distort}^{c})^{2}+(v_{distort}^{c})^{2}$.
The calculation happens in the camera frame (origin at principal point), so we must first convert the pixel coordinates, apply the correction, and then convert back.

In [6]:
# Define the variables
p_distort_I = np.array([400, 350])
a1 = 0.0001
a2 = 0.0000002
# o_x, o_y are from Q2

# 1. Convert distorted pixel coordinates to camera frame (origin at principal point)
u_distort_c = p_distort_I[0] - o_x
v_distort_c = p_distort_I[1] - o_y

# 2. Calculate r^2
r_sq = u_distort_c**2 + v_distort_c**2

# 3. Calculate the correction factor
correction_factor = 1 + a1 * r_sq + a2 * (r_sq**2)

# 4. Apply correction to find the undistorted coordinates in the camera frame
u_c_corrected = u_distort_c * correction_factor
v_c_corrected = v_distort_c * correction_factor

# 5. Convert corrected camera frame coordinates back to image pixel coordinates
u_I_corrected = u_c_corrected + o_x
v_I_corrected = v_c_corrected + o_y

print(f"Corrected pixel coordinates (u, v): [{u_I_corrected}, {v_I_corrected}]")
# Note: The distortion parameters are quite large, leading to a very significant correction.

Corrected pixel coordinates (u, v): [6024.0, 8083.0]
